In [1]:
import pathlib
import pandas as pd
import shutil
import logging

from idrstream.idr_stream import IdrStream

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target
0,LT0001_02,A4,50,4,positive control,ENSG00000149503
1,LT0001_02,A15,50,15,negative control,negative control
2,LT0001_02,B2,50,26,negative control,negative control
3,LT0001_02,C1,50,49,positive control,ENSG00000149503
4,LT0001_02,C4,50,52,positive control,ENSG00000149503
...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11
6739,LT0603_06,O16,50,352,negative control,negative control
6740,LT0603_06,P3,50,363,negative control,negative control
6741,LT0603_06,P21,50,381,positive control,COPB


In [3]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("tmp/")
final_data_dir = pathlib.Path("mitocheck_control_features/")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
    pass
except:
    print("No files to remove")

stream = IdrStream(idr_id, tmp_dir, final_data_dir, log='idr_stream2.log')

In [4]:
aspera_path = pathlib.Path("/home/roshankern/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [5]:
fiji_path = pathlib.Path("/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app")
stream.init_preprocessor(fiji_path)

[INFO] Overriding Leica ROI Reader; identifier: command:de.biovoxxel.utilities.RoiReader; jar: file:/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app/plugins/Biovoxxel_Plugins-2.5.6.jar


[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [8]:
stream.run_stream(data_to_process, batch_size=3, start_batch=0, batch_nums=[0, 1, 2], extra_metadata=["object_outlines"])

Completed: 75739K bytes transferred in 3 seconds
 (204638K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp2/downloads/LT0001_02/00004_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00004/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 87703K bytes transferred in 3 seconds
 (202993K bits/sec), in 1 file.
CellH5Reader initializing /home/roshankern/Desktop/Github/IDR_stream/tmp2/downloads/LT0001_02/00015_01.ch5
Plate :/sample/0/plate/
Well :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/
Site :/sample/0/plate/LT0001_02--ex2005_11_16--sp2005_02_17--tt17--c3/experiment/00015/position/
Parse segmentation ROIs for cell object primary__test : 0
Completed: 74464K bytes transferred in 3 seconds
 (197408K bits/sec), in 1 file.
CellH5Reader initiali

2022-10-14 14:17:22.684547: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
/home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.3 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/roshankern/D

2022-10-14 14:17:23,815 - WARNING - From /home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form tmp2/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'ENSG000001495

2022-10-14 14:17:25.497175: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-10-14 14:17:25.520514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-14 14:17:25.520967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2022-10-14 14:17:25.520981: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-14 14:17:25.523290: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-14 14:17:25.523320: I tensorflow/stream_executor/plat

2022-10-14 14:17:26,748 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream_prototype/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

...

2022-10-14 14:19:09,008 - WARNING - From /home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
Reading metadata form tmp2/DP_project/inputs/metadata/index.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Metadata_Plate  3 non-null      object
 1   Metadata_Well   3 non-null      object
 2   Metadata_Site   3 non-null      int64 
 3   Plate_Map_Name  3 non-null      object
 4   DNA             3 non-null      object
 5   Gene            3 non-null      object
 6   Gene_Replicate  3 non-null      int64 
dtypes: int64(2), object(5)
memory usage: 296.0+ bytes
None
{'negative cont

2022-10-14 14:19:10.667463: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-10-14 14:19:10.688680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-14 14:19:10.689141: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2022-10-14 14:19:10.689153: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-10-14 14:19:10.691440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-10-14 14:19:10.691469: I tensorflow/stream_executor/plat

2022-10-14 14:19:11,928 - WARNING - From /home/roshankern/Desktop/Github/IDR_stream_prototype/DeepProfiler/deepprofiler/learning/profiling.py:36: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input[0][0]                      
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________

/home/roshankern/anaconda3/envs/IDR_stream/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-10-14 14:19:12.621936: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-10-14 14:19:13.216906: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8401
2022-10-14 14:19:13.908212: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-14 14:19:13.908243: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-10-14 14:19:13.908998: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or dire

LT0001_02/74_50-1 (299 cells) : 2.55 secs
LT0001_02/304_50-1 (173 cells) : 0.50 secs
LT0001_02/315_50-1 (151 cells) : 0.48 secs
Profiling: done
0      [[311   5]\n [310   6]\n [309   6]\n [308   7]...
1      [[822  10]\n [821  11]\n [820  11]\n [819  11]...
2      [[110  12]\n [109  13]\n [108  13]\n [107  13]...
3      [[905  14]\n [904  15]\n [903  15]\n [902  16]...
4      [[266  17]\n [265  18]\n [264  18]\n [263  18]...
                             ...                        
294    [[1195  986]\n [1194  987]\n [1193  987]\n [11...
295    [[ 485  992]\n [ 484  993]\n [ 483  993]\n [ 4...
296    [[1228  993]\n [1227  994]\n [1226  994]\n [12...
297    [[ 171  996]\n [ 170  997]\n [ 169  997]\n [ 1...
298    [[ 486 1008]\n [ 485 1009]\n [ 484 1009]\n [ 4...
Name: object_outline, Length: 299, dtype: object
0      [[798   7]\n [797   8]\n [796   8]\n [795   8]...
1      [[853   7]\n [852   8]\n [851   9]\n [851  10]...
2      [[910   7]\n [909   8]\n [908   8]\n [907   8]...
3      [[